In [1]:
# dataframe management
import pandas as pd             

# numerical computation
import numpy as np

# visualization library
import seaborn as sns
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})   


# import matplotlib and allow it to plot inline
import matplotlib.pyplot as plt
%matplotlib inline

# seaborn can generate several warnings, we ignore them
import warnings 
warnings.filterwarnings("ignore")

from bokeh.layouts import gridplot
from bokeh.plotting import figure

from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
output_notebook()
from datetime import datetime, timedelta
from datetime import date
from scipy.stats import skew

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, ElasticNet, Lasso, LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor


Loading BokehJS ...

# Functions definitions

In [2]:
def toDate(x):
    parts = [int(el) for el in x.split("/")]
    return date(parts[2], parts[1], parts[0])

def r2_cv(model, sales_train, y, random_state=12345678):
    r2= cross_val_score(model, sales_train, y, scoring="r2", cv =KFold(10, shuffle=True, random_state=random_state)) 
    return(r2)

def rmse_cv(model, sales_train, y, random_state=12345678):
    rmse= np.sqrt(-cross_val_score(model, sales_train, y, scoring="neg_mean_squared_error", cv =KFold(10, shuffle=True, random_state=random_state)))
    return(rmse)

def RegionError(region, data):
    d = data[data[region] == 1][["StoreID","NumberOfActualSales","NumberOfSales"]].groupby(["StoreID"]).agg("sum")
    res = abs(d["NumberOfActualSales"]-d["NumberOfSales"]).agg("sum")
    return res / d["NumberOfActualSales"].agg("sum")

def q(col, quant, f):
    t = sales[col].quantile(quant)
    print(f'col {col} at {quant}-th quantile => {t}')
    sales.loc[f(sales[col], t), col] = t

# Read input data

In [3]:
#reading the notebook
sales = sales_string_date=pd.read_csv('train.csv')

#Removing tuples where stores are closed
sales = sales[sales['IsOpen'] == 1]

#converting to string columns that are labels
for el in ["StoreID", "Region"]:
    sales[el] = sales[el].apply(lambda x : str(x))

# Imputation of Missing Values

In [4]:
unknown_per_columns = sales.isnull().sum()

We see that Max_Gust_SpeedKm_h has 409947 missing values. We decided not to impute it.
Let's start with imputation of "Events"

In [5]:
null_Events = sales['Events'].isnull()
event_missing = sales[null_Events]
event_missing.describe()

,IsHoliday,IsOpen,HasPromotions,NearestCompetitor,NumberOfCustomers,NumberOfSales,Region_AreaKM2,Region_GDP,Region_PopulationK,CloudCover,...,Mean_TemperatureC,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees
count,103661.000000,103661.0,103661.000000,103661.000000,103661.000000,103661.000000,103661.000000,103661.000000,103661.000000,80397.000000,...,103661.000000,94779.000000,103661.000000,103661.000000,103661.000000,103661.000000,103661.000000,94779.000000,103661.000000,103661.000000
mean,0.001042,1.0,0.459179,8601.390619,314.641910,4893.577276,11435.271770,13711.770926,3467.389500,4.989340,...,11.545769,15.176421,11.082558,3.092774,40.196747,1015.171096,6.551538,10.586121,0.003920,134.812292
std,0.032261,0.0,0.498333,12222.596970,159.283348,2238.517426,8284.862826,2736.202628,2694.401824,2.233223,...,7.492758,6.648346,5.112519,6.093297,18.033663,7.151478,6.591203,6.138289,0.087722,103.812782
min,0.000000,1.0,0.000000,47.000000,0.000000,0.000000,344.000000,9893.000000,816.000000,0.000000,...,-11.000000,2.000000,3.000000,-17.000000,10.000000,991.000000,-12.000000,0.000000,0.000000,-1.000000
25%,0.000000,1.0,0.000000,1116.000000,215.000000,3375.000000,7215.000000,11849.000000,1293.000000,3.000000,...,6.000000,10.000000,6.000000,-2.000000,25.000000,1011.000000,1.000000,8.000000,0.000000,46.000000
50%,0.000000,1.0,0.000000,3614.000000,280.000000,4461.000000,9337.000000,13155.000000,1892.000000,5.000000,...,12.000000,13.000000,10.000000,3.000000,37.000000,1015.000000,7.000000,10.000000,0.000000,124.000000
75%,0.000000,1.0,1.000000,10593.000000,372.000000,5918.000000,15566.000000,15931.000000,5727.000000,7.000000,...,18.000000,19.000000,13.000000,8.000000,52.000000,1020.000000,12.000000,10.000000,0.000000,216.000000
max,1.000000,1.0,1.000000,85070.000000,2206.000000,26641.000000,32221.000000,23931.000000,8146.000000,8.000000,...,30.000000,31.000000,34.000000,18.000000,100.000000,1038.000000,23.000000,31.000000,4.060000,360.000000


In [6]:
sales['Events'].value_counts()

Rain                           171189
Fog                             56323
Fog-Rain                        41804
Rain-Thunderstorm               18116
Rain-Snow                       14576
Snow                            12541
Fog-Rain-Thunderstorm            6545
Fog-Snow                         2469
Fog-Rain-Snow                    2364
Thunderstorm                     1298
Rain-Hail                        1148
Rain-Snow-Hail                    513
Fog-Rain-Snow-Hail                231
Rain-Snow-Hail-Thunderstorm       193
Fog-Rain-Hail-Thunderstorm        178
Fog-Snow-Hail                     169
Fog-Rain-Hail                     168
Fog-Thunderstorm                  157
Snow-Hail                         134
Rain-Snow-Thunderstorm            131
Rain-Hail-Thunderstorm             50
Name: Events, dtype: int64

By this, we discovered that when Event is null the weather is good, from the fact that Precipitationmm mean is almost 0.
Furthermore, all the labels of Events are related to bad weather, that means that when no precipitation occurs the label is null.
We will impute Event by replacing the missing values with "Not Specified" (later).

Now we impute "CloudCover", making a distinction when it misses along with Events and when it misses on its own.

In [7]:
null_Events = sales['Events'].isnull()
null_CloudCover = sales['CloudCover'].isnull()
cloudcover_missing = sales[(null_Events)]
null_Events = sales['Events'].isnull()
event_missing = sales[null_Events]
event_cc_missing = sales[null_CloudCover & null_Events]
cloudcover_missing.shape, event_missing.shape, event_cc_missing.shape 

((103661, 36), (103661, 36), (23264, 36))

There are 28k tuples where both "Events" and "CloudCover" are missing, that means that the weather should be good.
for the remaining (41k-28k) (i.e. where "Events" is not null!) tuples we impute the CloudCoverage.

In [8]:
event_notmissing_cc_missing = sales[null_CloudCover & ~null_Events]
#computing mean where CloudCover is not null
mean_CC = sales["CloudCover"].mean()
event_notmissing_cc_missing["CloudCover"] = event_notmissing_cc_missing["CloudCover"].fillna(mean_CC)
sales = pd.concat([sales[~null_CloudCover | null_Events], event_notmissing_cc_missing])

Then, we impute the remaining rows with CloudCover missing

In [ ]:
null_Events = sales['Events'].isnull()
CloudyButNotEvent = sales[null_Events]
#there are some tuples with no Events but with the attribute CloudCover

In [ ]:
null_cloudCover = sales[sales['CloudCover'].isnull()]
mean_cloudCover = sales["CloudCover"].mean()
null_cloudCover["CloudCover"] = null_cloudCover["CloudCover"].fillna(mean_cloudCover)

sales = pd.concat([sales[~sales['CloudCover'].isnull()], null_cloudCover])

Now, we impute min,max,mean_VisibilityKm. 

In [ ]:
# checking if are all the same rows where visibility data are missing => yes, they are
sales[["Max_VisibilityKm", "Mean_VisibilityKm", "Min_VisibilitykM"]].count()

In [ ]:
null_visibility = sales[sales['Max_VisibilityKm'].isnull()]
mean_vis_max = sales["Max_VisibilityKm"].mean()
mean_vis_mean = sales["Mean_VisibilityKm"].mean()
mean_vis_min = sales["Min_VisibilitykM"].mean()

null_visibility["Max_VisibilityKm"] = null_visibility["Max_VisibilityKm"].fillna(mean_vis_max)
null_visibility["Mean_VisibilityKm"] = null_visibility["Mean_VisibilityKm"].fillna(mean_vis_mean)
null_visibility["Min_VisibilitykM"] = null_visibility["Min_VisibilitykM"].fillna(mean_vis_min)

sales = pd.concat([sales[~sales['Max_VisibilityKm'].isnull()], null_visibility])

Finally, we impute "Events".

In [ ]:
sales=sales.replace(np.nan,'NotSpecified', regex=True)


# Dealing with outliers

In [ ]:
sales.quantile(.99).sort_values(ascending=False).head(20)

In [ ]:
q("NearestCompetitor", .95, lambda x, y: x > y)
q("Precipitationmm", .99, lambda x, y: x > y)
q("Max_Wind_SpeedKm_h", .99, lambda x,y: x > y)
q("Max_Wind_SpeedKm_h", .01, lambda x,y: x < y)
q("Max_TemperatureC", .99, lambda x,y: x > y)
q("Max_TemperatureC", .01, lambda x,y: x < y)
q("Min_TemperatureC", .99, lambda x,y: x > y)
q("Min_TemperatureC", .01, lambda x,y: x < y)
q("Mean_Dew_PointC", .98, lambda x,y: x > y)
q("Mean_Dew_PointC", .05, lambda x,y: x < y)
q("Mean_Dew_PointC", .98, lambda x,y: x > y)
q("Mean_Dew_PointC", .05, lambda x,y: x < y)
q("Mean_Humidity", .95, lambda x,y: x > y)
q("Mean_Humidity", .01, lambda x,y: x < y)
q("Min_VisibilitykM", .98, lambda x,y: x > y)
q("Min_Humidity", .02, lambda x,y: x < y)
q("Min_Humidity", .98, lambda x,y: x > y)
q("Mean_Wind_SpeedKm_h", .99, lambda x,y: x > y)
q("Mean_TemperatureC", .01, lambda x,y: x < y)
q("Mean_TemperatureC", .99, lambda x,y: x > y)
q("Mean_VisibilityKm", .01, lambda x,y: x < y)
q("Mean_VisibilityKm", .99, lambda x,y: x > y)

In [ ]:
#Min_Sea_Level_PressurehPa  ->dropparla
#WindDirDegrees             -> inutile secondo me -> o la si categorizza in nord sud est ovest e la si fa con One Hot Encoding
#x=""
#sales[x].describe()
#sales[x].quantile(.01)
#sales[x].quantile(.95)

# Normalization of Numerical Variables

In [ ]:
# take the numerical features
numeric_feats = sales.dtypes[sales.dtypes != "object"].index
# compute the skewness but only for non missing variables (we already imputed them but just in case ...)
skewed_feats = sales[numeric_feats].apply(lambda x: skew(x.dropna()))

skewness = pd.DataFrame({"Variable":skewed_feats.index, "Skewness":skewed_feats.data})
# select the variables with a skewness above a certain threshold
skewness = skewness.sort_values('Skewness', ascending=[0])
f, ax = plt.subplots(figsize=(8,6))
plt.xticks(rotation='90')
sns.barplot(x=skewness['Variable'], y=skewness['Skewness'])
plt.ylim(0,25)
plt.xlabel('Numerical Variables', fontsize=15)
plt.ylabel('Skewness', fontsize=15)
plt.title('', fontsize=15)

# NORMALIZATION HERE

In [ ]:
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.drop(['IsHoliday','NumberOfCustomers', "NumberOfSales"])
sales[skewed_feats.index] = np.log1p(sales[skewed_feats.index])

# Correlation Analysis and Feature Selection

By analyzing the correlation, we see that the following variables do not add any additional information. 
Max_Dew_PointC, Min_Dew_PointC, Max_Sea_Level_PressurehPa, Mean_Sea_Level_PressurehPa, Max_Gust_SpeedKm_h
Finally, we drop 'NumberOfCustomers' because is not present in the submission dataset.

In [ ]:
sales=sales.drop(columns=['Max_Dew_PointC','Min_Dew_PointC','Max_Sea_Level_PressurehPa','Mean_Sea_Level_PressurehPa','Max_Gust_SpeedKm_h'])
sales.shape

Drop of the row where IsOpen==0

In [ ]:
sales=sales[sales['IsOpen']==1]
sales=sales.drop(columns=['IsOpen'])

### Convert Date to weekday label

In [ ]:
sales["Date"] = sales["Date"].apply(toDate)
sales["Day_Of_Week"] = sales["Date"].astype("datetime64").dt.weekday_name

# Dummify variables

In [ ]:
sales = pd.get_dummies(sales, columns=['StoreType','Events','AssortmentType', "Region", "Day_Of_Week"])

# Train and Test Definition
Separating the last 2 months, and use those as a test set and comparing the total of the predicted values.

In [ ]:
start_train = date(2018, 1, 1)
train = sales[sales["Date"] - start_train < timedelta(0)]
test = sales[sales["Date"] - start_train > timedelta(days=1)]
train = train.drop(columns=["Date"])
test = test.drop(columns=["Date"])

# One Hot Encoding of train and set

# First try: MultipleLinear Regression for Region 3

We start by predicting the number of customers for Region 3

In [ ]:
Region_label = "Region_3"
train_region3= train.loc[train[Region_label] == 1]
test_region3= test.loc[test[Region_label] == 1] 
train_region3.shape , test_region3.shape

# STEP 1 : PREDICT THE NUMBER OF CUSTOMERS

In [ ]:
train_x_region3_step1 = train_region3.drop(columns=['NumberOfSales','NumberOfCustomers'])
test_x_region3_step1 = test_region3.drop(columns=['NumberOfSales','NumberOfCustomers'])

train_y_region3_step1 = pd.DataFrame(data = train_region3['NumberOfCustomers'])
test_y_region3_step1 = pd.DataFrame(data = test_region3['NumberOfCustomers']) 

### DecisionTreeRegressor

In [ ]:
model_simple = RandomForestRegressor()
model_simple.fit(train_x_region3_step1, train_y_region3_step1)
yp = model_simple.predict(test_x_region3_step1) #yp=predicted customers del test

# compute R2 for train and using crossvalidation
r2_simple_train = r2_score(test_y_region3_step1,yp)
r2_xval_simple =  r2_cv(model_simple, test_x_region3_step1, test_y_region3_step1)

# compute RMSE for train and using crossvalidation
rmse_simple_train = mean_squared_error(test_y_region3_step1,yp,multioutput='raw_values')
rmse_xval_simple =  rmse_cv(model_simple, test_x_region3_step1,test_y_region3_step1)

print("Linear Regression")
print("==================================================")
print("\t                  Train R2=%.3f"%(r2_simple_train))
print("\t10-fold Crossvalidation R2=%.3f"%(r2_xval_simple.mean()))
print("\t                  Train RMSE=%.3f"%(rmse_simple_train))
print("\t10-fold Crossvalidation RMSE=%.3f"%(rmse_xval_simple.mean()))

In [ ]:
check = pd.DataFrame(test_y_region3_step1)
check[Region_label] = 1
check["NumberOfActualSales"] = test_y_region3_step1[["NumberOfCustomers"]]
check["NumberOfSales"] = yp
check["StoreID"] = test_x_region3_step1[["StoreID"]]
RegionError(Region_label, check)

# STEP 2 : PREDICT THE NUMBER OF SALES

Train over all the regions

In [ ]:
train_x_step2 = train.drop(columns=['NumberOfSales'])
train_y_step2 = pd.DataFrame(data = train['NumberOfSales'])

Test still over region 3

In [ ]:
test_x_region3_step2 = test_region3.drop(columns=['NumberOfSales','NumberOfCustomers'])
test_x_region3_step2['NumberOfCustomers']= yp
test_y_region3_step2 = pd.DataFrame(data = test_region3['NumberOfSales']) 

We keep only the most correlated features with the sales.

In [ ]:
train_x_step2 = train_x_step2[['NumberOfCustomers','Region_AreaKM2','HasPromotions','IsHoliday','Region_GDP',Region_label,"StoreID"]]
test_x_region3_step2 = test_x_region3_step2[['NumberOfCustomers','Region_AreaKM2','HasPromotions','IsHoliday','Region_GDP', Region_label, "StoreID"]]

In [ ]:
model_simple_step2 = RandomForestRegressor()
model_simple_step2.fit(train_x_step2, train_y_step2)
yp2 = model_simple_step2.predict(test_x_region3_step2) #yp2= le sales predette alla fine

In [ ]:
# compute R2 for train and using crossvalidation
r2_simple_train = r2_score(test_y_region3_step2,yp2)
r2_xval_simple =  r2_cv(model_simple, test_x_region3_step2, test_y_region3_step2)

# compute RMSE for train and using crossvalidation
rmse_simple_train = mean_squared_error(test_y_region3_step2, yp2, multioutput='raw_values')
rmse_xval_simple =  rmse_cv(model_simple, test_x_region3_step2, test_y_region3_step2)

print("Linear Regression")
print("==================================================")
print("\t                  Train R2=%.3f"%(r2_simple_train))
print("\t10-fold Crossvalidation R2=%.3f"%(r2_xval_simple.mean()))
print("\t                  Train RMSE=%.3f"%(rmse_simple_train))
print("\t10-fold Crossvalidation RMSE=%.3f"%(rmse_xval_simple.mean()))

In [ ]:
check = pd.DataFrame(test_y_region3_step2)
check[Region_label] = 1
check["NumberOfActualSales"] = test_y_region3_step2[["NumberOfSales"]]
check["NumberOfSales"] = yp2
check["StoreID"] = test_x_region3_step2[["StoreID"]]
RegionError(Region_label, check)